In [1]:
import os, sys, shutil

from glob import glob

module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
from datetime import datetime
def return_first_scenario(path):
    chosen_subfolder = None
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_dir() and entry.name != '.DS_Store':
                chosen_subfolder = entry.name
                break
    if chosen_subfolder is None:
        raise ValueError("No subfolder found")
    return chosen_subfolder



def find_earliest_latest_dates(layout_path):
    """
    Find the earliest and latest dates in a layout folder.
    """
    earliest_date = None
    latest_date = None
    for filename in os.listdir(os.path.join(layout_path, "Weather_Data")):
        if filename.endswith(".txt"):
            with open(os.path.join(layout_path, "Weather_Data", filename), "r") as f:
                lines = f.readlines()
                first_date = lines[0][:15]
                last_date = lines[-1][:15]
                first_date = datetime.strptime(first_date, "%Y %m %d %H%M")
                last_date = datetime.strptime(last_date, "%Y %m %d %H%M")
                if earliest_date is None or first_date < earliest_date:
                    earliest_date = first_date
                if latest_date is None or last_date > latest_date:
                    latest_date = last_date
    return earliest_date, latest_date

In [ ]:
import os
import pandas as pd

# Root folder containing all layout folders
LAYOUTS_ROOT = "./WideDataset"

layout_dates = []

for layout_id in os.listdir(LAYOUTS_ROOT):
    print(f"Processing layout: {layout_id}")
    layout_path = os.path.join(LAYOUTS_ROOT, layout_id)
    if not os.path.isdir(layout_path):
        continue

    try:
        start_day, end_day = find_earliest_latest_dates(layout_path)
        layout_dates.append({
            "layout_id": layout_id,
            "start_day": start_day,
            "end_day": end_day
        })
    except Exception as e:
        print(f"Skipping {layout_id} due to error: {e}")

# Convert to DataFrame
df = pd.DataFrame(layout_dates)

# Sort by start_day
df = df.sort_values("start_day")

# Save to CSV
df.to_csv("layout_date_ranges.csv", index=False)

print("Saved layout date ranges to layout_date_ranges.csv")


Processing layout: .DS_Store
Processing layout: 0002
Processing layout: 0001
Saved layout date ranges to layout_date_ranges.csv
